In [1]:
import cv2, os
from tqdm import tqdm
import numpy as np

In [2]:
import tensorflow as tf
from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Configure which GPU(s) to use
    tf.config.experimental.set_visible_devices(gpus, 'GPU')
    # Enable memory growth for each GPU
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
model = Model(inputs=base_model.input, outputs=x)

In [ ]:
path = "dataset/train/"

trainX = []
trainY = []
for classes in tqdm(os.listdir(path)):
    class_ = path + classes
    for images in os.listdir(class_):
        try:
            image_path = class_ + '/' + images
            image = cv2.imread(image_path)
            image = cv2.resize(image,(224,224))
            image = np.expand_dims(image,axis=0)
            features = model.predict(image)[0]
            trainX.append(features)
            trainY.append(class_.split('/')[-1])
        except:
            continue

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
trainX = np.array(trainX)
trainY = np.array(trainY)

In [ ]:
path = "dataset/test/"

testX = []
testY = []
for classes in tqdm(os.listdir(path)):
    class_ = path + classes
    for images in os.listdir(class_):
        try:
            image_path = class_ + '/' + images
            image = cv2.imread(image_path)
            image = cv2.resize(image,(224,224))
            image = np.expand_dims(image,axis=0)
            features = model.predict(image)[0]
            testX.append(features)
            testY.append(class_.split('/')[-1])
        except:
            continue

In [ ]:
testX = np.array(testX)
testY = np.array(testY)

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
trainY = le.fit_transform(trainY)
testY = le.fit_transform(testY)
trainY = to_categorical(trainY, 6)
testY = to_categorical(testY, 6)
print(trainX.shape,testX.shape,trainY.shape,testY.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model1 = Sequential()

# Add the input layer
model1.add(Dense(units=512, activation='relu', input_dim=2048))

# Add more hidden layers
model1.add(Dense(units=256, activation='relu'))
model1.add(Dense(units=128, activation='relu'))

# Add the output layer
model1.add(Dense(units=6, activation='softmax'))
model1.summary()

In [ ]:
num_gpus = 2
multi_model = multi_gpu_model(model1, gpus=num_gpus)

from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', np.unique(trainY), trainY)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

In [ ]:
multi_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])#, class_weight=class_weight_dict)

In [ ]:
multi_model.fit(trainX, trainY, epochs=30, validation_data = (testX,testY), batch_size=128)

In [ ]:
multi_model.save("nfsw.h5")

In [ ]:
from tensorflow.keras.models import load_model

mymodel = load_model("nfsw.h5")
classes = open("classes.txt",'r').read().split('\n')[:-1]
print(classes)

In [ ]:
path = "dataset/validation/"

validY = []
predY = []
for class_path in tqdm(os.listdir(path)):
    class_ = path + class_path
    for images in os.listdir(class_):
        try:
            image_path = class_ + '/' + images
            image = cv2.imread(image_path)
            image = cv2.resize(image,(224,224))
            image = np.expand_dims(image,axis=0)
            features = model.predict(image)[0]
            features = np.expand_dims(features,axis=0)
            pred = mymodel.predict(features)[0]
            name = classes[np.argmax(pred)]
            predY.append(name)
            validY.append(class_.split('/')[2])
        except:
            print(image_path)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
len(validY), len(predY), validY[0], predY[0]

In [ ]:
true_labels = validY
predicted_labels = predY
confusion_matrix = np.zeros((len(true_labels), len(true_labels)), dtype=np.int32)
labels = classes

for true_label, predicted_label in zip(true_labels, predicted_labels):
    row_idx = labels.index(true_label)
    col_idx = labels.index(predicted_label)
    confusion_matrix[row_idx, col_idx] += 1

# Display the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()